In [3]:
import networkx as nx
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns
import logging
import numpy as np
import geopandas as gpd
from geopy.geocoders import Nominatim
from shapely.wkt import loads
from scipy.spatial.distance import cdist

import plotly.graph_objects as go

# Let's brag about digital twins ;)

import sgp_dt as dt


In [4]:
%load_ext autoreload # to play with the lib in real time
%autoreload 2

ModuleNotFoundError: No module named 'autoreload # to play with the lib in real time'

# Checking what we have in the network

In [ ]:
# For reference only, the geodataframe with all path
fname = '../data/s3.zip'
SLC = gpd.read_file(fname)
SLC

#### With some stats

In [ ]:
stats = SLC.groupby('fclass')['Length'].agg(['sum','count']).astype(int).reset_index()
stats.columns = ["network","length (m)","number of segments"]
stats

# Starting the streamlit

In [ ]:
def loadShp(path):
    # Reading the overall network
    G= nx.readwrite.nx_shp.read_shp(path)
    # Support to network identification
    df_nodes = pd.read_csv("data/SG_nodes.txt", index_col=0)
    # Some cleaning necessary for csv loadup
    dfNodes = pd.read_csv("data/dfNodes.csv.zip")
    dfNodes.Pos = dfNodes.Pos.apply(lambda x: eval(x))
    dfEdges = pd.read_csv("data/dfEdges.csv.zip") 
    dfEdges.geometry = dfEdges.geometry.apply(lambda x: loads(x.replace('\'', '')))
    # Additional layers  
    gTrees = gpd.read_file('data/sTrees.zip') 
    gLamps = gpd.read_file('data/sLamps.zip') 
    gPark = gpd.read_file('data/sParks.zip') 
    gCCTV = gpd.read_file('data/sCCTV.zip') 
    return G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV

# This call should be cached
G, df_nodes, dfNodes, dfEdges, gTrees, gLamps, gPark, gCCTV = loadShp("data/s3/StreetsLampsCCTVTrees.shp")

In [ ]:
security   = 1 
cctv_perf  = 5.0
lamps_perf = 5.0
trees_perf = 5.0
#Different values for different types for paths
footway     = 2.0
residential = 1.5 
steps       = 2.0

UseDummyPoints = True

if UseDummyPoints:
    start,end = dt.getStartEnd("nop", "end", df_nodes, dummy= UseDummyPoints)
else:
    start,end = dt.getStartEnd(start_point, end_point, df_nodes, dummy= UseDummyPoints)

In [ ]:
weighted_G, pos, labels = dt.get_weighted_graph(G,security=1,cctv_perf=cctv_perf,lamps_perf=lamps_perf)

In [ ]:
dt.mapIt(start,end,weighted_G,dfNodes,dfEdges)

In [ ]:
def get_weighted_graphTrees(G,security=1,cctv_perf=1.0,lamps_perf=1.0,trees_perf=1.0):
    """
    Takes a graph G as input and adds the weights
    """
    weighted_G = nx.Graph()
    for data in G.edges(data=True):
        data=data
        coord1 = data[0]
        coord2 = data[1]

        # ZE formula to tweak
        data[2]['weight'] = data[2]['Length']
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['CCTV50mRE'] * cctv_perf )
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['Lamps50m']  * lamps_perf) 
        data[2]['weight'] = data[2]['weight'] / (1.0 + 0.1*data[2]['Trees10m']  * trees_perf) 

        # It is adapted for pedestrians ?
        relativeEase = 1.0
        if data[2]['fclass'] == "pedestrian":
            relativeEase = 0.3
        elif data[2]['fclass'] == "footway":
            relativeEase = 0.3
        elif data[2]['fclass'] == "steps":
            relativeEase = 0.6            
        data[2]['weight'] = data[2]['weight'] * relativeEase

        weighted_G.add_edge(coord1,coord2,weight=data[2]['weight'])

    return weighted_G

security   = 1.0
cctv_perf  = 1.0
lamps_perf = 1.0
trees_perf = 1.0

weighted_G = get_weighted_graphTrees(G,security=1,cctv_perf=cctv_perf,lamps_perf=lamps_perf,trees_perf=trees_perf)

In [ ]:
dt.mapIt(start,end,weighted_G,dfNodes,dfEdges)